In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small")
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small")

tokenizer_config.json:   0%|          | 0.00/259 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/497k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/277k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/840k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/903 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/457M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [5]:
import torch

def count_parameters_and_memory(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_memory = sum(p.numel() * p.element_size() for p in model.parameters() if p.requires_grad)
    return total_params, total_memory

def format_memory_size(bytes_size):
    # Convert bytes to MB or GB
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024

# Assuming 'model' is the variable holding your LLM
model_parameters, model_memory = count_parameters_and_memory(model)
formatted_memory = format_memory_size(model_memory)

print(f"The model has {model_parameters:,} parameters.")
print(f"The model is taking approximately {formatted_memory} of memory.")


The model has 111,008,256 parameters.
The model is taking approximately 423.46 MB of memory.


# reducing prescision

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small")

# Load model in float16
model1 = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small", torch_dtype=torch.float16)

def count_parameters_and_memory(model1):
    total_params = sum(p.numel() for p in model1.parameters() if p.requires_grad)
    total_memory = sum(p.numel() * p.element_size() for p in model1.parameters() if p.requires_grad)
    return total_params, total_memory

def format_memory_size(bytes_size):
    # Convert bytes to MB or GB
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024

# Assuming 'model' is the variable holding your LLM
model_parameters, model_memory = count_parameters_and_memory(model1)
formatted_memory = format_memory_size(model_memory)

print(f"The model has {model_parameters:,} parameters.")
print(f"The model is taking approximately {formatted_memory} of memory.")


The model has 111,008,256 parameters.
The model is taking approximately 211.73 MB of memory.


# applying QLoRA

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import LoraConfig, get_peft_model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small")

# Load model in float16
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small", torch_dtype=torch.float16)

# Apply dynamic quantization
model = torch.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8  # dtype of quantized weights
)

# Define LoRA configuration
lora_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=64,
      bias="none",
      task_type="CAUSAL_LM",
)
LoraConfig()
# Apply LoRA to the model
model = get_peft_model(model, lora_config)

def count_parameters_and_memory(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_memory = sum(p.numel() * p.element_size() for p in model.parameters() if p.requires_grad)
    return total_params, total_memory

def format_memory_size(bytes_size):
    # Convert bytes to MB or GB
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024

# Calculate the number of parameters and memory usage of the quantized model
model_parameters, model_memory = count_parameters_and_memory(model)
formatted_memory = format_memory_size(model_memory)

print(f"The LoRA-adapted and quantized model has {model_parameters:,} parameters.")
print(f"The LoRA-adapted and quantized model is taking approximately {formatted_memory} of memory.")


The LoRA-adapted and quantized model has 2,359,296 parameters.
The LoRA-adapted and quantized model is taking approximately 9.00 MB of memory.


/opt/conda/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1091: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


# reducing rank of model

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import LoraConfig, get_peft_model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small")

# Load model in float16
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small", torch_dtype=torch.float16)

# Apply dynamic quantization
model = torch.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8  # dtype of quantized weights
)

# Define LoRA configuration
lora_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=32,
      bias="none",
      task_type="CAUSAL_LM",
)
LoraConfig()
# Apply LoRA to the model
model = get_peft_model(model, lora_config)

def count_parameters_and_memory(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_memory = sum(p.numel() * p.element_size() for p in model.parameters() if p.requires_grad)
    return total_params, total_memory

def format_memory_size(bytes_size):
    # Convert bytes to MB or GB
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024

# Calculate the number of parameters and memory usage of the quantized model
model_parameters, model_memory = count_parameters_and_memory(model)
formatted_memory = format_memory_size(model_memory)

print(f"The LoRA-adapted and quantized model has {model_parameters:,} parameters.")
print(f"The LoRA-adapted and quantized model is taking approximately {formatted_memory} of memory.")


The LoRA-adapted and quantized model has 1,179,648 parameters.
The LoRA-adapted and quantized model is taking approximately 4.50 MB of memory.


In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import LoraConfig, get_peft_model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small")

# Load model in float16
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small", torch_dtype=torch.float16)

# Apply dynamic quantization
model = torch.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8  # dtype of quantized weights
)

# Define LoRA configuration
lora_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=8,
      bias="none",
      task_type="CAUSAL_LM",
)
LoraConfig()
# Apply LoRA to the model
model = get_peft_model(model, lora_config)

def count_parameters_and_memory(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_memory = sum(p.numel() * p.element_size() for p in model.parameters() if p.requires_grad)
    return total_params, total_memory

def format_memory_size(bytes_size):
    # Convert bytes to MB or GB
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024

# Calculate the number of parameters and memory usage of the quantized model
model_parameters, model_memory = count_parameters_and_memory(model)
formatted_memory = format_memory_size(model_memory)

print(f"The LoRA-adapted and quantized model has {model_parameters:,} parameters.")
print(f"The LoRA-adapted and quantized model is taking approximately {formatted_memory} of memory.")


The LoRA-adapted and quantized model has 294,912 parameters.
The LoRA-adapted and quantized model is taking approximately 1.12 MB of memory.


# rank=4

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from peft import LoraConfig, get_peft_model

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("codeparrot/codeparrot-small")

# Load model in float16
model = AutoModelForCausalLM.from_pretrained("codeparrot/codeparrot-small", torch_dtype=torch.float16)

# Apply dynamic quantization
model = torch.quantization.quantize_dynamic(
    model,  # the original model
    {torch.nn.Linear},  # a set of layers to dynamically quantize
    dtype=torch.qint8  # dtype of quantized weights
)

# Define LoRA configuration
lora_config = LoraConfig(
      lora_alpha=16,
      lora_dropout=0.1,
      r=4,
      bias="none",
      task_type="CAUSAL_LM",
)
LoraConfig()
# Apply LoRA to the model
model = get_peft_model(model, lora_config)

def count_parameters_and_memory(model):
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_memory = sum(p.numel() * p.element_size() for p in model.parameters() if p.requires_grad)
    return total_params, total_memory

def format_memory_size(bytes_size):
    # Convert bytes to MB or GB
    for unit in ['B', 'KB', 'MB', 'GB', 'TB']:
        if bytes_size < 1024:
            return f"{bytes_size:.2f} {unit}"
        bytes_size /= 1024

# Calculate the number of parameters and memory usage of the quantized model
model_parameters, model_memory = count_parameters_and_memory(model)
formatted_memory = format_memory_size(model_memory)

print(f"The LoRA-adapted and quantized model has {model_parameters:,} parameters.")
print(f"The LoRA-adapted and quantized model is taking approximately {formatted_memory} of memory.")


The LoRA-adapted and quantized model has 147,456 parameters.
The LoRA-adapted and quantized model is taking approximately 576.00 KB of memory.
